In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import os
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import rasterio
from statsmodels.tsa.seasonal import STL
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime as dt
from matplotlib.colors import TwoSlopeNorm 
import time
import sys

import rasterio
from rasterstats import zonal_stats
from rasterio.plot import show
import multiprocessing

In [3]:
#fire_data_classified = gpd.read_file('../../Classification_Fires/fires_data_classified.geojson')
#Select only the agricultural fires
#fire_data = fire_data_classified[fire_data_classified['agricultural'] == 1]

shape = gpd.read_file('../../../../../../src/data_preprocessing/base_geojson/TL_state_shapefile_for_clip.geojson')
shape = shape.explode()

district = gpd.read_file('../../../../../../src/data_preprocessing/base_geojson/TS_district_boundary.json')
district = district.reset_index()

mandal = gpd.read_file('../../../../../../src/data_preprocessing/base_geojson/TS_mandal_boundary.json')
mandal = mandal.reset_index()

boundary = gpd.read_file('../../../../../../src/data_preprocessing/base_geojson/Telangana_grid1km.geojson')
boundary = boundary.reset_index()


In [4]:
viirs = pd.read_csv('../../../../../../../UNDP/Fire/data/VIIRS1.csv')

In [5]:
fire_pts = gpd.GeoDataFrame(viirs,                        #Converting fire points into dataframe
                        geometry=gpd.points_from_xy(
                            viirs.longitude,
                            viirs.latitude),
                        crs=4326)

In [6]:
geo_fire_data = fire_pts[['acq_date','geometry']]
geo_fire_data = geo_fire_data.sort_values(by=["acq_date"])
geo_fire_data['acq_date'] = pd.to_datetime(geo_fire_data['acq_date'])
geo_fire_data = geo_fire_data.clip(boundary)
geo_fire_data['fireID'] = [a for a in range(0, len(geo_fire_data))]

polygons = []

#-------create a buffer of square of 1km size using buffer function from shapely-----#

for i in geo_fire_data.geometry:
    p1 = i
    buffer = p1.buffer(0.001689189, cap_style = 3)         #187.5m = 0.001689189 and cap_style 3 is square box of same of 375m side length
    polygons.append(buffer)

#create a new column in GeoDataFrame newdf and dump polygon buffer of respective point values
geo_fire_data['geometry buffered'] = polygons 

geo_fire_data['acq_date'] =  pd.to_datetime(geo_fire_data['acq_date'])
geo_fire_data['year'] = (geo_fire_data['acq_date']).dt.year

geo_fire_data = geo_fire_data[geo_fire_data['year']<2022]
geo_fire_data

    

,acq_date,geometry,fireID,geometry buffered,year
2829527,2021-05-08,POINT (80.35246 16.85648),0,"POLYGON ((80.35414418900001 16.858165189, 80.3...",2021
808260,2018-03-18,POINT (80.35403 16.86220),1,"POLYGON ((80.355716189 16.863887189, 80.355716...",2018
2531446,2021-03-23,POINT (80.35525 16.88592),2,"POLYGON ((80.35693618900001 16.887609189, 80.3...",2021
1977891,2020-04-19,POINT (80.33079 16.89140),3,"POLYGON ((80.332484189 16.893085189, 80.332484...",2020
2755220,2021-04-16,POINT (80.32949 16.89159),4,"POLYGON ((80.33118018900001 16.893279189, 80.3...",2021
...,...,...,...,...,...
119545,2017-03-08,POINT (77.53736 18.34765),144726,"POLYGON ((77.53905018900001 18.349338189, 77.5...",2017
2512051,2021-03-21,POINT (77.52328 18.35765),144727,"POLYGON ((77.524966189 18.359334189000002, 77....",2021
1874156,2020-03-27,POINT (77.53098 18.36233),144728,"POLYGON ((77.53267218900001 18.364018189, 77.5...",2020
147869,2017-03-19,POINT (77.52982 18.44215),144729,"POLYGON ((77.531505189 18.443841189, 77.531505...",2017


In [7]:
LULC_location = '../../../../../../src/data_preprocessing/LULC/'
df = pd.DataFrame(columns=geo_fire_data.columns)

for a in geo_fire_data['year'].unique():

    year_data = geo_fire_data[geo_fire_data['year'] == a]
    tiff = LULC_location+'01-01-'+str(a)+'.tif'
    lulc = rasterio.open(tiff, mode = 'r')
    lulc_array = lulc.read(1) # landuse corresponding to each rasterpixel, so we extracted the pixel values from the raster
    # affine: 1: corresponds to the width of each pixel, 2: row rotation, 3: x-coordinate of the upper left pixel, 4: column rotation, 5: height of each pixel, 6: y-coordinate of the upper left pixel
    affine = lulc.transform
    cmap = {1: 'Water', 2: 'Trees', 4: 'Flooded Vegetation', 5: 'Crops', 7: 'Built Area', 8: 'Bare Ground', 9: 'Snow/Ice', 10: 'Clouds', 11: 'Rangeland'}
    year_data['Land use'] = 0
    year_data['Trees Around'] = 0

    for i in range(0, len(year_data)):
        test = zonal_stats(year_data['geometry'].iloc[i], lulc_array, affine = affine, geojson_out = True, stats = 'majority', nodata = lulc.nodata, categorical=True, category_map = cmap, all_touched=True)
        try: 
            year_data['Land use'].iloc[i] = test[0]['properties']['majority']
        except:
            year_data['Land use'].iloc[i] = 'unknown'

    for i in range(0, len(year_data)):
        test = zonal_stats(year_data['geometry buffered'].iloc[i], lulc_array, affine = affine, geojson_out = True, stats = 'majority', nodata = lulc.nodata, categorical=True, category_map = cmap, all_touched=True)

        if 'Trees' in list(test[0]['properties'].keys()):
            year_data['Trees Around'].iloc[i]=1
        else:
            year_data['Trees Around'].iloc[i]=0

    
    df = pd.concat([df, year_data], axis = 0)

In [8]:
df['agricultural'] = 0
df['agricultural strict'] = 0

for i in range(0, len(df)):
    if (df['Land use'].iloc[i] == 5) | (df['Land use'].iloc[i] == 4) :
        df['agricultural'].iloc[i] = 1
    if ((df['Land use'].iloc[i] == 5) | (df['Land use'].iloc[i] == 4)) & (df['Trees Around'].iloc[i]==0):
        df['agricultural strict'].iloc[i] = 1

In [9]:
df = gpd.GeoDataFrame(df[['fireID', 'acq_date','agricultural', 'agricultural strict', 'geometry']])

In [10]:
df.to_file('fires_data_classified.geojson', driver="GeoJSON")

In [4]:
df = gpd.read_file('fires_data_classified.geojson')

In [113]:
boundaries = boundary
beginyear = 2017
endyear = 2021
'''
#Make sure the geometry columns are in the right format
geo_fire_data = df[['geometry', 'acq_date', 'fireID']]
geo_fire_data['geometry'] = geo_fire_data['geometry'].to_crs(epsg = 4326)
geo_fire_data['acq_date'] =  pd.to_datetime(geo_fire_data['acq_date'])
geo_fire_data['year'] = (geo_fire_data['acq_date']).dt.year
geo_fire_data['month'] = (geo_fire_data['acq_date']).dt.month
geo_fire_data['day'] = (geo_fire_data['acq_date']).dt.day
geo_fire_data = geo_fire_data[(geo_fire_data['acq_date'] >= str(beginyear)+'-01-01') & (geo_fire_data['acq_date'] < str(endyear+1)+'-01-01')]
fires_per_boundaries= gpd.sjoin(geo_fire_data, boundaries, how="inner")
'''
geo_fire_data['geometry'] = geo_fire_data['geometry'].to_crs(epsg = 4326)
boundaries['geometry'] = boundaries['geometry'].to_crs(epsg = 4326)

#Load date into date format
geo_fire_data['acq_date'] =  pd.to_datetime(geo_fire_data['acq_date'])
geo_fire_data['year'] = (geo_fire_data['acq_date']).dt.year
geo_fire_data['month'] = (geo_fire_data['acq_date']).dt.month
geo_fire_data['day'] = (geo_fire_data['acq_date']).dt.day

#Selects the years we are interested in, depending on the input of the function
geo_fire_data = geo_fire_data[(geo_fire_data['acq_date'] >= str(beginyear)+'-01-01') & (geo_fire_data['acq_date'] < str(endyear+1)+'-01-01')]

#Count all fires within a region given by the boundaries dataframe
fires_per_boundaries= gpd.sjoin(geo_fire_data, boundaries, how="inner")

#Create the right time format: We count per month
fires_per_boundaries['day'] = 1 
fires_per_boundaries['year'] = pd.Series(pd.to_numeric(fires_per_boundaries['year'], errors='coerce'), dtype='int64')
fires_per_boundaries['month'] = pd.Series(pd.to_numeric(fires_per_boundaries['month'], errors='coerce'), dtype='int64')
fires_per_boundaries['ModifiedDateTime'] = pd.to_datetime(fires_per_boundaries[['year', 'month', 'day']].astype('int64').astype('str'), yearfirst=True)

#Sum amount of fires per mandal per month per year make sure that if no fire happens at a specific time write a zero
fires_per_boundaries_count = fires_per_boundaries.groupby(['index', 'ModifiedDateTime'])['fireID'].count().unstack(fill_value=0).stack().reset_index()
fires_per_boundaries_count['Fires'] = fires_per_boundaries_count[0] 
del fires_per_boundaries_count[0]


In [ ]:
dates_ordinal = [dt.datetime.toordinal(i) for i in fires_per_boundaries_count[fires_per_boundaries_count['index'] == 0]['ModifiedDateTime']]

indices = [i for i in fires_per_boundaries_count['index'].unique()]

In [139]:
def Trend_Score(i):

    df = fires_per_boundaries_count[fires_per_boundaries_count['index'] == i]
    df = pd.Series(list(df['Fires']), index=pd.to_datetime(df['ModifiedDateTime']), name="Fires")
    stl = STL(df)

    res = stl.fit()

    #Set the data in the right format for Linear Regression
    x = np.array(dates_ordinal)
    X = x.reshape(-1, 1)
    y = np.array(res.trend)
    y = y.reshape(-1, 1)

    #Perform Linear Regression and obtain the slope
    reg = LinearRegression().fit(X, y)
    y_pred_trend = reg.predict(X)
    slope, intercept = np.polyfit(x, y_pred_trend,1)
    return slope[0]

In [159]:
start_time = time.time()


pool = multiprocessing.Pool()

#func = partial(ndvi_dppd, boundary, tiffs, directory)

results = [result for result in pool.map(Trend_Score, fires_per_boundaries_count['index'].unique())]
#pool.map(func, iterable)
pool.close()
pool.join()


print("--- %s seconds ---" % (time.time() - start_time))



--- 121.11800622940063 seconds ---


In [182]:
res = np.array(results)
data_norm = np.where(res > 0, res/np.max(res), -res/np.min(res))
data_norm = data_norm* -1  # negatives values means decrease trend in fire hence it is positive deviance

In [183]:
df = pd.DataFrame()
df['index'] = indices
df['deviances'] = data_norm

In [184]:
gdf = df.merge(boundary, on='index', how='left')

In [185]:
gpd.GeoDataFrame(gdf).to_file('fire_deviance.json')

In [5]:
gdf = gpd.read_file('fire_deviance.json')

In [10]:
print(gdf['geometry'][0])

POLYGON ((77.24690974997284 16.45502054967031, 77.24673599623769 16.464041072140393, 77.2373878002232 16.46387314973425, 77.23756198423618 16.454852724337336, 77.24690974997284 16.45502054967031))
